In [1]:
import sys
import pandera as pa
import hvplot
import hvplot.pandas  # noqa
import panel as pn
import polars as pl
from dataclasses import dataclass
from hplc_py.pipeline.preprocess_dashboard import DataSets, PreProcesser
from pandera.typing.polars import LazyFrame, DataFrame
import pybeads as be

%reload_ext autoreload
%autoreload 2

pl.Config.set_tbl_rows(10)
pn.extension()
pn.config.comms = "vscode"
sys.executable

dsets = DataSets()
ringland = dsets.ringland.fetch()
ringland.head()

,detection,color,varietal,id,code_wine,time,signal
i64,str,str,str,str,str,f64,f64
0,"""raw""","""red""","""shiraz""","""7b085f32-4d69-…","""a0301_2021 chr…",0.005833,0.001952
1,"""raw""","""red""","""shiraz""","""7b085f32-4d69-…","""a0301_2021 chr…",0.0125,0.001825
2,"""raw""","""red""","""shiraz""","""7b085f32-4d69-…","""a0301_2021 chr…",0.019167,0.002004
3,"""raw""","""red""","""shiraz""","""7b085f32-4d69-…","""a0301_2021 chr…",0.025833,0.002861
4,"""raw""","""red""","""shiraz""","""7b085f32-4d69-…","""a0301_2021 chr…",0.0325,0.003964


# Baseline Correction Via BEADS

See: http://www.laurent-duval.eu/Articles/Ning_X_2014_j-chemometr-intell-lab-syst_chromatogram_bedusbeads-preprint.pdf and https://github.com/skotaro/pybeads

[text](preprocesing_1_smoothing_sharpening.ipynb)

In [43]:
import matplotlib.pyplot as plt

y = ringland.select("signal").to_series(0).to_numpy().ravel()

# try changing fc and lam0-2, amp if these dont' fit your data
# fc = 0.006

"""
y: Noisy observation.
d: Filter order (d = 1 or 2).
fc: Filter cut-off frequency (cycles/sample) (0 < fc < 0.5).
r: Asymmetry ratio for penalty function.
Nit: Number of iteration (usually 10 to 30 is enough).
lam0, lam1, lam2: Regularization parameters.
pen: Penalty function, 'L1_v1' or 'L1_v2'.
"""
# decreasing fc decreases fit
fc = 6e-3
d = 1
r = 6
amp = 0.8
lam0 = 0.5 * amp
lam1 = 5 * amp
lam2 = 4 * amp
Nit = 15
pen = "L1_v2"

signal_est, bg_est, cost = be.beads(y, d, fc, r, Nit, lam0, lam1, lam2, pen, conv=None)

df = pl.DataFrame({"signal": y, "signal_corr": signal_est, "bg": bg_est})
df.with_row_index("idx").plot(
    x="idx",
    y=["signal_corr", "signal", "bg"],
    grid=True,
    xlim=(310, 1450),
    ylim=(-10, 50),
)

:NdOverlay   [Variable]
   :Curve   [idx]   (value)

Conclusion: As of 2024/04/09, this package is too undocumented for continued use. Dont know how to select parameters to produce a desired fit.